<a href="https://colab.research.google.com/github/tae-min-lee/t-m-po/blob/master/22lipnet_light_%EC%88%98%EC%A0%95%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.layers import Lambda

from tensorflow.keras.layers import BatchNormalization, GRU, Conv3D, ZeroPadding3D,MaxPooling3D,Dense, Activation, SpatialDropout3D, Flatten, Input, Bidirectional, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

import copy, cv2, os, random, glob, shutil
import numpy as np

# Prepare

In [ ]:
 import copy, cv2, os, random, glob, shutil
 import numpy as np

 video_root = r'./train_dataset/video' # video파일이 있는 주소
 align_root =  r'./train_dataset/align' # align파일이 있는 주소
 video_dirlist = glob.glob(video_root + '/*') # video 디렉토리를 읽어온다

 train_align_path = []
 train_video_path = []

 for k in range(len(video_dirlist)):
     print(video_dirlist[k])
     video_path_ACEN = []
    video_ids = os.listdir(video_dirlist[k])# 각 speaker들의 디렉토리를 가져온다
    # print(video_ids)
    
     for video_id in video_ids:
         video_path_ACEN = glob.glob(video_dirlist[k]+'/'+video_id + '/*.png')# 각speaker들이 말한 이미지 파일을 읽어온다
         train_video_path.append(video_path_ACEN)# train_video에 추가를 한ㄴ다
         train_align_path.append(align_root+'/'+video_id+'.align')# align파일을 추가한다

./train_dataset/video\s10
./train_dataset/video\s11
./train_dataset/video\s12
./train_dataset/video\s13
./train_dataset/video\s14
./train_dataset/video\s15
./train_dataset/video\s16
./train_dataset/video\s17
./train_dataset/video\s18
./train_dataset/video\s19
./train_dataset/video\s23
./train_dataset/video\s24
./train_dataset/video\s25
./train_dataset/video\s26
./train_dataset/video\s27
./train_dataset/video\s28
./train_dataset/video\s29
./train_dataset/video\s3
./train_dataset/video\s30
./train_dataset/video\s31
./train_dataset/video\s32
./train_dataset/video\s33
./train_dataset/video\s34
./train_dataset/video\s4
./train_dataset/video\s5
./train_dataset/video\s6


In [ ]:

 video_root = r'./val_dataset/video'
 align_root =  r'./train_dataset/align'
 video_dirlist = glob.glob(video_root + '/*')

 val_align_path = []
 val_video_path = []

 for k in range(len(video_dirlist)):
     print(video_dirlist[k])
     video_path_ACEN = []
          video_ids = os.listdir(video_dirlist[k])
    # print(video_ids)
    
     for video_id in video_ids:
         video_path_ACEN = glob.glob(video_dirlist[k]+'/'+video_id + '/*.png')
         val_video_path.append(video_path_ACEN)
         val_align_path.append(align_root+'/'+video_id+'.align')

./val_dataset/video\s1
./val_dataset/video\s2
./val_dataset/video\s20
./val_dataset/video\s22


## save pickle

In [ ]:
# import pickle
# name = './train_video_path.pickle'
# with open(name,'wb') as fw:
#     pickle.dump(train_video_path, fw)
    
# name = './train_align_path.pickle'
# with open(name,'wb') as fw:
#     pickle.dump(train_align_path, fw)
    
# name = './val_video_path.pickle'
# with open(name,'wb') as fw:
#     pickle.dump(val_video_path, fw)
    
# name = './val_align_path.pickle'
# with open(name,'wb') as fw:
#     pickle.dump(val_align_path, fw)

## load pickle

In [ ]:
import pickle # 파이썬 객체 자체를 파일로 저장하는 것
name = './train_video_path.pickle'
with open(name,'rb') as fw:
    train_video_path = pickle.load(fw)
    
name = './train_align_path.pickle'
with open(name,'rb') as fw:
    train_align_path = pickle.load(fw)
    
name = './val_video_path.pickle'
with open(name,'rb') as fw:
    val_video_path = pickle.load(fw)
    
name = './val_align_path.pickle'
with open(name,'rb') as fw:
    val_align_path = pickle.load(fw)

In [ ]:
print(train_align_path[8082][22:-6])
print(train_video_path[8081-1][0][:-20])

bgir9p
./train_dataset/video\s18/


## 폴더속 이미지의 크기가 75가 아닌경우는 모두 삭제

In [ ]:
# for check in range(len(train_video_path)):
#     if len(train_video_path[check])!=75:
#         print(train_align_path[check])
#         print(train_video_path[check-1][0])
#         shutil.rmtree(train_video_path[check-1][0][:-20]+train_align_path[check][22:-6])
#         print()

In [ ]:
len(val_align_path),len(val_video_path)

(3964, 3964)

## Datagenerator

In [ ]:
def text_to_labels(text): # text를 label로 변환하는 함수
    ret = []
    for char in text:
        if char >= 'a' and char <= 'z': # 만약 텍스트가 a에서 z사이인 문자라면 
            ret.append(ord(char) - ord('a'))# 그 택스트를 아스키코드 값으로 바꾸고 a의 아스키코드 값을 빼준다
        elif char == ' ':# 만약 공백이라면
            ret.append(26) #26을 추가해준다
    return ret

def labels_to_text(labels):# label을 text로 바꿔주는 함수
    # 26 is space, 27 is CTC blank char
    text = ''
    for c in labels:
        if c >= 0 and c < 26:
            text += chr(c + ord('a'))# 아스키코드값을 사용하여 label값으로 (문자)로 바꿔준다
        elif c == 26: #26이라면 blank로 바꿔준다
            text += ' '
    return text

In [ ]:
import numpy as np

class Align(object):
    def __init__(self, absolute_max_string_len=32, label_func=None):
        self.label_func = label_func
        self.absolute_max_string_len = absolute_max_string_len

    def from_file(self, path):
        with open(path, 'r') as f:
            lines = f.readlines()
        align = [(int(y[0])/1000, int(y[1])/1000, y[2]) for y in [x.strip().split(" ") for x in lines]]
        self.build(align)
        return self

    def from_array(self, align):
        self.build(align)
        return self

    def build(self, align):
        self.align = self.strip(align, ['sp','sil'])
        self.sentence = self.get_sentence(align)
        self.label = self.get_label(self.sentence)
        self.padded_label = self.get_padded_label(self.label)

    def strip(self, align, items):
        return [sub for sub in align if sub[2] not in items]

    def get_sentence(self, align):
        return " ".join([y[-1] for y in align if y[-1] not in ['sp', 'sil']])

    def get_label(self, sentence):
        return self.label_func(sentence)

    def get_padded_label(self, label):
        padding = np.ones((self.absolute_max_string_len-len(label))) * -1
        return np.concatenate((np.array(label), padding), axis=0)

    @property
    def word_length(self):
        return len(self.sentence.split(" "))

    @property
    def sentence_length(self):
        return len(self.sentence)

    @property
    def label_length(self):
        return len(self.label)


In [ ]:
def data_generator(video,align,batch_size = 32) :

 
    
    video_path = video # 각각 video와 align을 받는다
    align_path = align
    #random.shuffle(idx_list)

    while 1: # 무한루프
        idx = 0
        idx_list = list(range(len(video_path))) # 비디오의 갯수를 길이로 숫자로 list를 만든다

        batch_imgs = []
        batch_labs = []
        label_length = []
        input_length = []
        source_str = []

        if idx > len(idx_list) - batch_size:# batch size를 뺸 값이 idx보다 클 때
            tmp_list =idx_list[idx:] #idx_list의 idx부터 끝까지를 tmp_list로 넣는다
            idx = 0
        else: #아니라면
            tmp_list = idx_list[idx:idx+batch_size]# idx부터 batch size를 더한만큼을 tmp_list에 넣는다
            idx = idx + batch_size

        for i in tmp_list:

            imgs=[]
            for v in video_path[i]:# batch_size만큼부터 차례로 cv2를 이용하여 이미지를 읽어온다
                img = cv2.imread(v)/255
                imgs.append(img)# 읽어온 이미지를 imgs에 추가한다

            align = Align(32, text_to_labels).from_file(align_path[i]) # align도 배치 크기 많큼 가져온다

            batch_labs.append(align.padded_label)
            batch_imgs.append(np.array(imgs))
            #source_str.append(align.sentence)
            label_length.append(align.label_length)
            input_length.append(len(imgs))

        #source_str = np.array(source_str)
        label_length = np.array(label_length) #align에 대한것
        input_length = np.array(input_length)# img에 대한것
        batch_labs = np.array(batch_labs)
        batch_imgs = np.array(batch_imgs)

        inputs = [batch_imgs, batch_labs, input_length, label_length]

        outputs =  np.zeros([batch_size])  # dummy data for dummy loss function

        yield inputs, outputs


In [ ]:
# #random.shuffle(idx_list)
# video_path = train_video_path
# align_path = train_align_path

# dddddd=1
# while 1:
#     idx = 0
#     batch_imgs = []
#     batch_labs = []
#     batch_size = 64
#     label_length = []
#     input_length = []
#     source_str = []



#     idx_list = list(range(len(video_path)))
#     #random.shuffle(idx_list)

#     batch_imgs = []
#     batch_labs = []
#     label_length = []
#     input_length = []
#     source_str = []

#     if idx > len(idx_list) - batch_size:
#         tmp_list =idx_list[idx:]
#         idx = 0
#     else:
#         tmp_list = idx_list[idx:idx+batch_size]
#         idx = idx + batch_size

#     for i in tmp_list:

#         imgs=[]
#         for v in video_path[i]:
#             img = cv2.imread(v)
#             imgs.append(img)

#         align = Align(32, text_to_labels).from_file(align_path[i])

#         batch_labs.append(align.padded_label)
#         batch_imgs.append(imgs)
#         source_str.append(align.sentence)
#         label_length.append(align.label_length)
#         input_length.append(len(imgs))

#     source_str = np.array(source_str)
#     label_length = np.array(label_length)
#     input_length = np.array(input_length)
#     batch_labs = np.array(batch_labs)
#     batch_imgs = np.array(batch_imgs)

#     inputs = {'the_input': batch_imgs,
#               'the_labels': batch_labs,
#               'input_length': input_length,
#               'label_length': label_length,
#               'source_str': source_str
#               }

#     outputs =  np.zeros([batch_size]) 

#     print(dddddd)
#     print(inputs['the_input'].shape)
#     print(inputs['the_labels'].shape)
#     print(inputs['input_length'].shape)
#     print(inputs['label_length'].shape)
#     print(outputs.shape)
#     dddddd=dddddd+1
    

# CTC

In [ ]:
def CTC(name, args):
	return Lambda(ctc_lambda_func, output_shape=(1,), name=name)(args)

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # From Keras example image_ocr.py:
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    # y_pred = y_pred[:, 2:, :]
    y_pred = y_pred[:, :, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Model

In [ ]:
class LipNet(object):
    def __init__(self, img_c=3, img_w=100, img_h=50, frames_n=75, absolute_max_string_len=32, output_size=28):
        self.img_c = img_c
        self.img_w = img_w
        self.img_h = img_h
        self.frames_n = frames_n
        self.absolute_max_string_len = absolute_max_string_len
        self.output_size = output_size
        self.build()

    def build(self):
        input_shape = (self.frames_n, self.img_h, self.img_w, self.img_c)

        self.input_data = Input(name='the_input', shape=input_shape, dtype='float32')

        self.zero1 = ZeroPadding3D(padding=(1, 2, 2), name='zero1')(self.input_data)
        self.conv1 = Conv3D(32, (3, 5, 5), strides=(1, 2, 2), kernel_initializer='he_normal', name='conv1')(self.zero1)
        self.batc1 = BatchNormalization(name='batc1')(self.conv1)
        self.actv1 = Activation('relu', name='actv1')(self.batc1)
        self.drop1 = SpatialDropout3D(0.5)(self.actv1)
        self.maxp1 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')(self.drop1)

        self.zero2 = ZeroPadding3D(padding=(1, 2, 2), name='zero2')(self.maxp1)
        self.conv2 = Conv3D(64, (3, 5, 5), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv2')(self.zero2)
        self.batc2 = BatchNormalization(name='batc2')(self.conv2)
        self.actv2 = Activation('relu', name='actv2')(self.batc2)
        self.drop2 = SpatialDropout3D(0.5)(self.actv2)
        self.maxp2 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')(self.drop2)

        self.zero3 = ZeroPadding3D(padding=(1, 1, 1), name='zero3')(self.maxp2)
        self.conv3 = Conv3D(96, (3, 3, 3), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv3')(self.zero3)
        self.batc3 = BatchNormalization(name='batc3')(self.conv3)
        self.actv3 = Activation('relu', name='actv3')(self.batc3)
        self.drop3 = SpatialDropout3D(0.5)(self.actv3)
        self.maxp3 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')(self.drop3)

        self.resh1 = TimeDistributed(Flatten())(self.maxp3)

        self.gru_1 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'), merge_mode='concat')(self.resh1)
        self.gru_2 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru2'), merge_mode='concat')(self.gru_1)

        # transforms RNN output to character activations:
        self.dense1 = Dense(self.output_size, kernel_initializer='he_normal', name='dense1')(self.gru_2)

        self.y_pred = Activation('softmax', name='softmax')(self.dense1)

        self.labels = Input(name='the_labels', shape=[self.absolute_max_string_len], dtype='float32')
        self.input_length = Input(name='input_length', shape=[1], dtype='int64')
        self.label_length = Input(name='label_length', shape=[1], dtype='int64')

        self.loss_out = CTC('ctc', [self.y_pred, self.labels, self.input_length, self.label_length])

        self.model = Model(inputs=[self.input_data, self.labels, self.input_length, self.label_length], outputs=self.loss_out)

    def summary(self):
        Model(inputs=self.input_data, outputs=self.y_pred).summary()

    def predict(self, input_batch):
        return self.test_function([input_batch, 0])[0]  # the first 0 indicates test

    @property
    def test_function(self):
        # captures output of softmax so we can decode the output during visualization
        return K.function([self.input_data, K.learning_phase()], [self.y_pred, K.learning_phase()])

In [ ]:
lipnet = LipNet()

In [ ]:
lipnet.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, 75, 50, 100, 3)]  0         
_________________________________________________________________
zero1 (ZeroPadding3D)        (None, 77, 54, 104, 3)    0         
_________________________________________________________________
conv1 (Conv3D)               (None, 75, 25, 50, 32)    7232      
_________________________________________________________________
batc1 (BatchNormalization)   (None, 75, 25, 50, 32)    128       
_________________________________________________________________
actv1 (Activation)           (None, 75, 25, 50, 32)    0         
_________________________________________________________________
spatial_dropout3d (SpatialDr (None, 75, 25, 50, 32)    0         
_________________________________________________________________
max1 (MaxPooling3D)          (None, 75, 12, 25, 32)   

In [ ]:
# import tensorflow as tf
# tf.test.is_gpu_available(
#     cuda_only=False, min_cuda_compute_capability=None
# )

# Train

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)

batch_size = 128

train_gen = data_generator(train_video_path,train_align_path,batch_size = batch_size)
val_gen = data_generator(val_video_path,val_align_path, batch_size=batch_size)

weight_path = './weights/lipnet_ep{epoch:03d}-{val_loss:.4f}.hdf5'

checkpoint_callback = ModelCheckpoint(filepath= weight_path,
                                  monitor='val_loss',
                                  save_best_only=1,
                                  mode='auto')

history = lipnet.model.fit_generator(generator =train_gen,
                              epochs = 100,
                              steps_per_epoch = len(train_video_path)/batch_size-1,
                              validation_data = val_gen,
                              validation_steps = len(val_video_path)/batch_size-1,
                              callbacks=checkpoint_callback
                              )


Epoch 1/100
 13/201 [>.............................] - ETA: 14:04 - loss: 71.4816

In [ ]:
print(train_gen.data_generator())